<a href="https://colab.research.google.com/github/Sanjayme049/ML/blob/main/spam_ham_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
# classification

# 1.Naive Bayes.
# 2. Decision Trees
# 3. Random Forest
# 4.Xgboost
# 5.Support vector Machine(SVM)

In [57]:
# Unsupervised
# 1.dimensionality
# 2.Clustering

# clustering
# 1 K-means
# Gmm
# spectaral



In [58]:
import pandas as pd
import numpy as np
import re


In [59]:
data= pd.read_csv('https://raw.githubusercontent.com/utsav-195/sms-spam-ham-classification/main/data/data_cleaned.csv', on_bad_lines='skip',delimiter=';')


In [60]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [61]:
data.head()

,class,message,not_stemmed,stemmed
0,ham,"Go until jurong point, crazy.. Available only ...",go until juron point crazy available only in b...,go juron point crazi avail bugi n great world ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking if you on,ok lar joke
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in number a wily comp to win fa cup...,free entri number wili comp win fa cup final t...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor you c already then say,u dun say earli hor c alreadi say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i don t think he goes to us he lives aroun...,nah think goe us live around though


In [62]:
# preprocessing
# 1.Remove stop  words & special characters
# 2.
# 3.limitised the words
# 

In [63]:
df= data.iloc[:, :2]

In [64]:
df

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [65]:
df.message = df.message.apply(lambda x: x.lower())


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [66]:
df.message= df.message.str.replace(r'\W+',' ', regex=True)
df.message= df.message.str.replace(r'\d+',' ', regex=True)

In [67]:
df.message = df.message.apply(lambda x: ' '.join([word for word in x.split() if word != ' ']))


In [68]:
import spacy
nlp= spacy.load('en_core_web_sm')


In [69]:
stopwords = nlp.Defaults.stop_words
df.message = df.message.apply(lambda x: ' '.join([ i.lemma_ for i in nlp(x) if i.text not in stopwords]))

In [70]:
df.message


0       jurong point crazy available bugis n great wor...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                                     u dun early hor u c
4                             nah don t think go usf live
                              ...                        
5567    nd time try contact u u win pound prize claim ...
5568                             ü b go esplanade fr home
5569                                 pity mood suggestion
5570    guy bitching act like d interested buy week gi...
5571                                            rofl true
Name: message, Length: 5572, dtype: object

In [71]:
# Now we have data in clean format, we are ready to apply Tfi-idf
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(max_features=1000)
tfidf_df = vec.fit_transform(df.message)

In [72]:
# note: If not using tfidf_df.toarray()... we end up with this ERROR("cannot concatenate object of type '<class 'scipy.sparse._csr.csr_matrix'>'; only Series and DataFrame objs are valid")


In [73]:
# To see the dataframe after Tfidf
tfidf_df = pd.DataFrame(tfidf_df.toarray() , columns = vec.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [74]:
tfidf_df.head(2)

,abiola,able,abt,ac,accept,access,account,actually,ad,add,...,yahoo,yar,yeah,year,yep,yes,yesterday,yo,yr,yup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
new_df = df.rename(columns={'class' : 'label_col'})

In [76]:
new_df.head(2)

,label_col,message
0,ham,jurong point crazy available bugis n great wor...
1,ham,ok lar joking wif u oni


In [77]:
# Concate the Label to the tfidf_df
ml_data = pd.concat([tfidf_df , new_df.label_col] , axis=1)

In [78]:
ml_data.head(3)

,abiola,able,abt,ac,accept,access,account,actually,ad,add,...,yar,yeah,year,yep,yes,yesterday,yo,yr,yup,label_col
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ham
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spam


In [79]:
from sklearn.model_selection import train_test_split

# Apply the train test split
X = tfidf_df
y = new_df.label_col

X_train , X_test , y_train , y_test = train_test_split( X , y , test_size = 0.2)

In [80]:
# Applying Gausian Naive Bayes
from sklearn.naive_bayes import GaussianNB
modelG = GaussianNB()
modelG.fit(X_train, y_train)

GaussianNB()

In [81]:
# Accuracy of Gausian Naive Bayes 
print("Accuracy of Gausian Naive Bayes: ",modelG.score(X_test, y_test))


Accuracy of Gausian Naive Bayes:  0.7713004484304933


In [82]:
# OR WAY TO CALCULATE THE ACCURACY SCORE


In [83]:
# Accuracy of Gausian Naive Bayes
from sklearn.metrics import accuracy_score
y_pred = modelG.predict(X_test)
score = accuracy_score(y_test , y_pred)
print("Accuracy of Gausian Naive Bayes:",score)

Accuracy of Gausian Naive Bayes: 0.7713004484304933


In [84]:
# Predicting the accuracy of gaussian  naive bayes using cross validation
from sklearn.model_selection import cross_val_score
score = cross_val_score(modelG,X,y,cv=5)
print("Cross validation score for  gaussian naive bayes:",score.mean())

Cross validation score for  gaussian naive bayes: 0.7704594601122284


In [85]:
# Applying Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
modelM = MultinomialNB()
modelM.fit(X_train, y_train)


MultinomialNB()

In [86]:
# Accuracy of Multinomial Naive Bayes 
print("Accuracy of Multinomial Naive Bayes: ",modelM.score(X_test, y_test))

Accuracy of Multinomial Naive Bayes:  0.9802690582959641


In [87]:
# OR WAY TO CALCULATE THE ACCURACY SCORE


In [88]:
# Accuracy of Multinomial Naive Bayes
from sklearn.metrics import accuracy_score
y_pred = modelM.predict(X_test)
score = accuracy_score(y_test , y_pred)
print("Accuracy of Multinomial Naive Bayes:",score)


Accuracy of Multinomial Naive Bayes: 0.9802690582959641


In [89]:
# Predicting the accuracy of multinomial  naive bayes using cross validation
from sklearn.model_selection import cross_val_score
score = cross_val_score(modelM,X,y,cv=5)
print("Cross validation score for  multinomial naive bayes:",score.mean())

Cross validation score for  multinomial naive bayes: 0.9797187044625678
